In [1]:
import numpy as np
import gzip, pickle
import pandas as pd

patients = pd.read_csv('internacoes_charlson_zero.csv.gz', compression='gzip', \
                       nrows=None, usecols=['target'])
target = patients['target'].values
patients.shape

(48907, 1)

In [2]:
grams = {}

with gzip.open("data_5k_unigram.npy.gz", "rb") as wfp: 
    grams['unigram'] = pickle.load(wfp)
    wfp.close()
print(grams['unigram'].shape)
    
with gzip.open("data_5k_bigram.npy.gz", "rb") as wfp: 
    grams['bigram'] = pickle.load(wfp)
    wfp.close()
print(grams['bigram'].shape)
    
with gzip.open("data_5k_trigram.npy.gz", "rb") as wfp: 
    grams['trigram'] = pickle.load(wfp)
    wfp.close()
print(grams['trigram'].shape)
    
with gzip.open("data_10k_multigram.npy.gz", "rb") as wfp: 
    grams['multigram'] = pickle.load(wfp)
    wfp.close()    
print(grams['multigram'].shape)

(48907, 5000)
(48907, 5000)
(48907, 5000)
(48907, 10000)


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

import sys

features = 5000
step = 250
ranges = range(step,features+1,step)

for k in ranges:
    sys.stdout.write(str(k)+'\t')
    
    for i, gram in enumerate(grams.keys()):
        data = grams[gram]    
        
        kfold1 = StratifiedKFold(n_splits=2)
        for i1, (train, test) in enumerate(kfold1.split(data, target)):
            break

        data2 = data[test]
        target2 = target[test]
        
        score = cross_val_score(RandomForestRegressor(n_jobs=5), data2[:,:k], target2, \
                                 n_jobs=1,cv=6, scoring='neg_mean_absolute_error', verbose=0)

        sys.stdout.write(str(round(abs(np.mean(score)),2))+'\t')
    
    sys.stdout.write('\n')

250	2.24	

KeyboardInterrupt: 